In [71]:
import time
import requests
import pandas as pd
import numpy as np
api_token='____'

In [72]:
# Парсим сайт с валютами и заполняем DataFrame

url='https://cbr.ru/currency_base/daily/'
query=requests.get(url,verify=False).text

start=query.find('<th>Курс</th>\r\n        </tr>\r\n        <tr>\r\n          <td>')
length=len('<th>Курс</th>\r\n        </tr>\r\n        <tr>\r\n          <td>')
stop=query.find('</tr>\r\n      </tbody>\r\n    </table>\r\n')
df=query[start+length:stop].split('</td>\r\n        </tr>\r\n        <tr>\r\n          <td>')

df=list(map(lambda x:x.replace('</th>\r\n','').\
    replace('<th>','').\
    replace('<td>','').\
    replace('</td>\r\n',''), df))
df=list(map(lambda x: x.replace('ких','кий').replace('ов',''),df))
df=list(map(lambda x: list(filter(None, x.split('  '))), df))



data = pd.DataFrame(columns=['id','code','value','name', 'rate'])
for i in range(len(df)):
    df[i][-1]=df[i][-1].replace(',','.')
    data.loc[i]=df[i]


data = data.astype({'rate': float, 'value': int})

data['rate_per_1']=data['rate']/data['value']

data



C:\Users\belovvs\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cbr.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,id,code,value,name,rate,rate_per_1
0,036,AUD,1,Австралийский доллар,52.8442,52.844200
1,944,AZN,1,Азербайджанский манат,42.2941,42.294100
2,051,AMD,100,Армянский драм,15.0298,0.150298
3,933,BYN,1,Белорусский рубль,29.0170,29.017000
4,975,BGN,1,Болгарский лев,42.4591,42.459100
5,986,BRL,1,Бразильский реал,12.9871,12.987100
6,348,HUF,100,Венгерский форинт,23.0698,0.230698
7,410,KRW,1000,Вон Республики Корея,60.3187,0.060319
8,344,HKD,10,Гонконгский доллар,92.3573,9.235730
9,208,DKK,1,Датская крона,11.1599,11.159900


In [75]:
data.to_csv('data.csv', index=False)

In [67]:
import time

currency = (data['code']+'-'+data['name']).to_list()
currency_text=''
for i in currency:
    currency_text += i + '\n'

currency_name = (data['name'].apply(lambda x: x.lower())).to_list()
currency_code = (data['code'].apply(lambda x: x.lower())).to_list()
cur = currency_name + currency_code


from typing import Optional

def get_updates(offset_param: Optional[str] = None):
    api_url = 'https://api.telegram.org/bot'+api_token+'/getUpdates'
    if offset_param == None:
        response = requests.get(api_url).json()
    else:
        response = requests.get(api_url, params = {'offset' : offset_param}).json()
    return response['result']

def send_message(chat_id, message):
    api_url = "https://api.telegram.org/bot"+api_token+"/sendMessage?"
    requests.get(api_url, params={'chat_id':chat_id,\
                                 'text':message})

def check_message(chat_id, message):
    for_exchange = message.split(',')
    
    if message =='/start':
        text = 'Приветсвуем, мы показываем курсы валют.\n'+'Имеющиеся валюты:\n'+'\n'+currency_text+'Введите название или код валюты (Например, USD или Австралийский доллар)'
        text2 ='Можно прислать сумму,которую хочется конвертировать в рубли. В формате: сумма,код валюты (пример: 1000,USD)'
        send_message(chat_id, text+'. '+text2)
       
    elif message.lower() in cur:
        for i in range(len(data)):
            if (data.loc[i]['code'].lower() == message.lower()) or (data.loc[i]['name'].lower() == message.lower()):
                print('Отправил курс')
                rate = data.loc[i]['rate_per_1']
                send_message(chat_id, f'Курс {rate}')
    elif len(for_exchange) > 1:
        if (message.split(',')[1].lower() in currency_code):
            for i in range(len(data)):
                if data.loc[i]['code'].lower() == message.split(',')[1].lower():
                    print('Отправил расчет')
                    rate = data.loc[i]['rate_per_1']
                    try:
                        send_message(chat_id, for_exchange[0] + ' ' + for_exchange[1].upper() + '=' + str(round(float(message.split(',')[0])*float(rate),2))+' RUB')
                    except:
                        send_message(chat_id, 'Не корректно введено')
        else:
            send_message(chat_id,'Не корректно введено')
        
    else:
        print('кто-то ввел не корректно валюту')
        send_message(chat_id, 'Есть только такие валюты:' + '\n' + currency_text)
            
  
            
def run():
    last_update_id = get_updates()[-1]['update_id'] # Присваиваем ID последнего отправленного сообщения боту
    offset_param = last_update_id
    while True:
        print('Update')
        time.sleep(1)
        messages = get_updates(offset_param) # Получаем обновления
        for message in messages:
            # Если в обновлении есть ID больше чем ID последнего сообщения, значит пришло новое сообщение
            if last_update_id < message['update_id']:
                last_update_id = message['update_id'] # Присваиваем ID последнего отправленного сообщения боту
                chat_id = message['message']['chat']['id'] #запоминаем chat_id
                text = message['message']['text'] # запоминаем присланый текст
                check_message(chat_id, text) # Проверяем, что написано в сообщении

In [ ]:
run()